In [14]:
from osgeo import gdal
import os



# Reproject and clip input rasters

In [15]:

def openRaster(raster):
    closeOnExit = False
    try:
        raster.GetProjection()
        openRaster = raster
    except AttributeError:
        openRaster = gdal.Open(raster)
        closeOnExit = True
    return openRaster, closeOnExit



def getRasterInfo(raster):

    r, closeOnExit = openRaster(raster)
    proj = r.GetProjection()
    gt = r.GetGeoTransform()
    sizeX = r.RasterXSize
    sizeY = r.RasterYSize
    extent = [gt[0], gt[3]+gt[5]*sizeY, gt[0]+gt[1]*sizeX, gt[3]]
    bands = r.RasterCount
    if closeOnExit:
        r = None
  
    return proj, gt, sizeX, sizeY, extent, 

def resampleWithGdalWarp(srcFile, templateFile, outFile="", outFormat="MEM",
                         resampleAlg="near"):
    # Get template projection, extent and resolution
    proj, gt, sizeX, sizeY, extent= getRasterInfo(templateFile)
    # Resample with GDAL warp
    outDs = gdal.Warp(outFile,
                      srcFile,
                      format=outFormat,
                      dstSRS=proj,
                      xRes=gt[1],
                      yRes=gt[5],
                      outputBounds=extent,
                      resampleAlg=resampleAlg)
    print(outFile)

In [17]:
template = "/nas02/kegro/dk-sentinel-2/2022/interpolated/32VNH/B08/S2_32VNH_20220105_B08.tif"
raster = "/nas02/sulova/13_innoforest/World_Cover_32VNH.tif"
output_file = os.path.join(os.path.dirname(raster), '{}_near.tif'.format(os.path.basename(raster).split('\\')[-1][:-4])) 

- Change to different method if does not fit your purpose: ResampleAlg="near"

In [8]:
resampleWithGdalWarp(raster, template, outFile=output_file, outFormat="GTiff", resampleAlg="near")

/nas02/sulova/13_innoforest/World_Cover_32VNH_align.tif


# Mask

 - Define the mask ouput file

In [50]:
input_raster =  output_file
output_file_mask = os.path.join(os.path.dirname(input_raster), '{}_mask.tif'.format(os.path.basename(input_raster).split('\\')[-1][:-4])) 

In [67]:
import rasterio as rst
import numpy as np

with rst.open(input_raster) as src:
        data = src.read(1)
        data = np.where(data==10, 1, data)
        profile = src.profile
        print(profile)
        profile.update(nodata = 99)
        print(profile)
        if output_file_mask != None:
            with rst.open(output_file_mask, 'w',  **profile) as dst:
                dst.write(data.astype(rst.uint8), 1)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 10980, 'height': 10980, 'count': 1, 'crs': CRS({'init': 'epsg:32632'}), 'transform': Affine(10.0, 0.0, 499980.0,
       0.0, -10.0, 6300000.0), 'tiled': False, 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 99, 'width': 10980, 'height': 10980, 'count': 1, 'crs': CRS({'init': 'epsg:32632'}), 'transform': Affine(10.0, 0.0, 499980.0,
       0.0, -10.0, 6300000.0), 'tiled': False, 'interleave': 'band'}
